In [1]:
from numerapi import NumerAPI, utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

napi = NumerAPI()

In [2]:
# set your parameters here

models = ["nyuton_test6", "nyuton_test4"] #, "nyuton_test7"]

starting_round = 314
end_round = 335

corr_multiplier = 1
tc_multiplier = 2

In [3]:
def getModelPerformance(model):
    model = napi.round_model_performances(model)
    m = pd.DataFrame(model)
    m = m[(m.roundNumber>=starting_round) & (m.roundNumber<=end_round)]
    m['result'] = m['corr'] * corr_multiplier + m['tc'] * tc_multiplier
    
    return m

df = pd.DataFrame()
for m in models:

    m1 = getModelPerformance(m)
    m1 = m1[['tc', 'corr']]
    m1.columns = [c+'_'+m for c in m1.columns]

    df = pd.concat([df, m1], axis=1)

df = df.dropna()
df

,tc_nyuton_test6,corr_nyuton_test6,tc_nyuton_test4,corr_nyuton_test4
13,0.107317,0.009054,0.092954,0.020954
14,0.010365,-0.002814,-0.036986,-0.010486
15,-0.058755,-0.005126,-0.005541,0.004106
16,0.031642,0.042601,0.008512,0.046518
17,0.036026,0.025968,0.022570,0.030869
18,0.057050,0.054669,0.004741,0.074832
19,0.040319,0.079121,0.035468,0.085959
20,0.057364,0.049950,0.091399,0.055057
21,-0.012178,0.032832,0.021568,0.010713
23,0.071746,0.041040,0.081641,0.021680


In [5]:
reordered_columns = df.columns.to_list()
corr_columns = [c for c in df.columns if 'corr_' in c]
tc_columns = [c for c in df.columns if 'tc_' in c]

#calculate the returns
mpo_data = pd.concat(
        [
            df[corr_columns] * corr_multiplier,
            df[tc_columns] * tc_multiplier
        ],
        axis=1
    )[reordered_columns] \
    .rolling(2, axis=1) \
    .sum()[reordered_columns[1::2]] \
    .rename( \
        columns={
            c: "return_" + m
            for c,m in zip(corr_columns, models)
        }
    )

def mpo_function(x, Rcov, R):
    
    portfolio_std = np.sqrt(x.reshape(1, -1).dot(Rcov).dot(x))
    portfolio_return = R.reshape(1, -1).dot(x)
    
    return - portfolio_return / portfolio_std

#covariance matrix of the returns
Rcov = mpo_data.cov().values

#return per model
R = mpo_data.mean(axis=0).values

result = minimize(
    mpo_function,
    args=(Rcov, R),
    x0=np.ones(len(models)),
    method='Nelder-Mead'
)
print("Minimization successful:", result.success)

# Normalize, weights should sum up to 1
result.x = result.x / result.x.sum()
for w, m in zip(result.x, models):
    print(m,': ', w)

Minimization successful: True
nyuton_test6 :  0.4118601557800669
nyuton_test4 :  0.5881398442199331


In [6]:
summary = []
for m in models:
    summary.append((m, mpo_data["return_"+m].mean(), mpo_data["return_"+m].std()))
    
portfolio = (mpo_data*result.x).sum(axis=1)
summary.append(('Portfolio', portfolio.mean(), portfolio.std()))

summary = pd.DataFrame(summary, columns = ['Model', 'Corr', 'STD'])
summary['Sharpe'] = summary['Corr'] / summary['STD']
summary

,Model,Corr,STD,Sharpe
0,nyuton_test6,0.086065,0.105928,0.812485
1,nyuton_test4,0.089299,0.100609,0.887582
2,Portfolio,0.087967,0.090737,0.969468
